## Homework 1

**使用第三方天气查询API，实现完整可执行的天气查询应用**

首先，定义3个函数`get_city_adcode`、`get_amap_weather`和`execute_function_call`
- get_city_adcode:用于获取城市及地区编码
- get_amap_weather：用于通过第三方天气查询api实现当日天气查询，并返回查询结果
- execute_function_call: 用于根据消息中的功能调用信息来执行相应的功能并获取结果

然后，创建一个消息列表，并向其中添加了一个系统消息和一个用户消息。系统消息的内容是指示对话的目标，用户消息的内容是用户的问题。

接着，使用`chat_completion_request`函数发出聊天请求并获取响应，然后从响应中提取出助手的消息并添加到消息列表中。

如果助手的消息中包含功能调用，那么就使用`execute_function_call`函数执行这个功能调用并获取结果，然后将结果作为一个功能消息添加到消息列表中。

最后，使用`pretty_print_conversation`函数打印出整个对话。

In [14]:
import json
import requests
import os
from tenacity import retry, wait_random_exponential, stop_after_attempt
from termcolor import colored

GPT_MODEL = "gpt-3.5-turbo"


#### 定义获取城市编码的函数get_city_adcode()

In [2]:
import requests

def get_city_adcode(city_name):
    """
    根据城市名称获取城市的adcode码。

    参数:
    city_name (str): 城市名称，例如 "北京市"
    api_key (str): 高德地图API密钥

    返回:
    str: 城市的adcode码
    """
    # 从环境变量中获取API密钥
    api_key = os.getenv('AMAP_API_KEY')
    if not api_key:
        return "Error: API key not found. Please set the AMAP_API_KEY environment variable."
    
    # 高德地图地理编码API的URL
    url = 'https://restapi.amap.com/v3/geocode/geo'

    # 构造请求参数
    params = {
        'key': api_key,
        'address': city_name,
        'output': 'json'
    }

    # 发送GET请求
    response = requests.get(url, params=params)
    
    # 检查请求是否成功
    if response.status_code == 200:
        json_data = response.json()
        
        # 检查API返回的状态码
        if json_data['status'] == '1' and json_data['count'] != '0':
            # 获取第一个匹配项的adcode
            adcode = json_data['geocodes'][0]['adcode']
            return adcode
        else:
            return "Error: API returned error status."
    else:
        return f"Error: HTTP request failed with status code {response.status_code}."

# 使用你的高德地图API密钥
city_name = '北京'
adcode = get_city_adcode(city_name)
print(f"\n地  区： {city_name}")
print(f"adcode: {adcode}\n")


地  区： 北京
adcode: 110000



#### 定义查询天气函数get_amap_weather()

In [10]:
def get_amap_weather(city, extensions="all"):
    """
    使用高德天气API查询给定城市的天气状况。
    
    :param city: 要查询天气的城市名称或者城市的adcode。
    :param extensions: 可选，'base' 返回实时天气，'all' 返回预报天气。
    :return: 包含天气信息的字典，如果出错则返回错误信息。
    """
    # 从环境变量中获取API密钥
    api_key = os.getenv('AMAP_API_KEY')
    if not api_key:
        return "Error: API key not found. Please set the AMAP_API_KEY environment variable."
    
    # 高德天气API的URL
    url = 'https://restapi.amap.com/v3/weather/weatherInfo'
    
    # 构造请求参数
    params = {
        'key': api_key,        # API密钥
        'city': city,          # 城市名称或adcode
        'extensions': extensions, # 'base': 实时天气；'all': 预报天气
        'output': 'json'       # 响应格式
    }
    
    try:
        # 发送GET请求
        response = requests.get(url, params=params)
        response.raise_for_status()  # 如果响应状态码不是200，将抛出HTTPError异常
        
        # 解析JSON数据
        weather_data = response.json()
        
        # 根据需要提取天气信息
        if weather_data.get('status') == '1':  # 请求成功
            if extensions == "base":
                # 实时天气信息处理
                live_data = weather_data.get('lives', [{}])[0]
                # 这里可以添加生成实时天气描述的代码
                return generate_weather_realtime_narrative(live_data)
            else:
                # 预报天气信息处理
                forecast_data = weather_data.get('forecasts', [{}])[0]
                return generate_weather_forecast_narrative(forecast_data)
        else:
            # 返回错误信息
            return f"Error: {weather_data.get('info')}"
        
    except requests.RequestException as e:
        return f"Network error: {e}"

# 对实时天气返回结果进行处理，将其处理为一段连续的文字：
def generate_weather_realtime_narrative(live_data):
    """
    根据提供的实时天气信息生成连续的文字描述。

    :param live_data: 包含实时天气信息的字典。
    :return: 实时天气的连续文字描述。
    """
    province = live_data.get('province', '指定省份')
    city = live_data.get('city', '指定城市')
    weather = live_data.get('weather', '未知天气')
    temperature = live_data.get('temperature', '未知温度')
    wind_direction = live_data.get('winddirection', '未知风向')
    wind_power = live_data.get('windpower', '未知风力')
    humidity = live_data.get('humidity', '未知湿度')
    report_time = live_data.get('reporttime', '未知时间')

    # 构建实时天气描述文本
    realtime_narrative = (f"{city}的实时天气更新于{report_time}。"
                          f"当前天气{weather}，温度为{temperature}度，"
                          f"{wind_direction}风，风力{wind_power}级，"
                          f"相对湿度{humidity}%。")

    return realtime_narrative


# 对预报天气返回结果进行处理，将其处理为一段连续的文字：
def generate_weather_forecast_narrative(forecast_data):
    """
    根据提供的天气预报信息生成连续的文字描述。
    
    :param forecast_data: 包含预报天气信息的字典。
    :return: 天气预报的连续文字描述。
    """
    province = forecast_data.get('province', '指定省份')
    city = forecast_data.get('city', '指定城市')
    report_time = forecast_data.get('reporttime', '指定时间')
    casts = forecast_data.get('casts', [])
    
    # 初始化描述文本
    forecast_narrative = f"{city}的天气预报，更新时间为{report_time}。未来几天的天气情况如下："

    # 对每一天的天气信息进行描述
    for cast in casts:
        date = cast.get('date')
        day_weather = cast.get('dayweather')
        night_weather = cast.get('nightweather')
        day_temp = cast.get('daytemp')
        night_temp = cast.get('nighttemp')
        day_wind = cast.get('daywind')
        night_wind = cast.get('nightwind')
        day_power = cast.get('daypower')
        night_power = cast.get('nightpower')

        forecast_narrative += (f"\n{date}，星期{cast.get('week')}，"
                               f"白天{day_weather}，气温{day_temp}度，"
                               f"{day_wind}风{day_power}级；"
                               f"夜间{night_weather}，气温{night_temp}度，"
                               f"{night_wind}风{night_power}级。")
    
    return forecast_narrative



# 使用函数示例
city = '110101'  # 城市adcode，例如北京市东城区或110101
weather_info = get_amap_weather(city, extensions="base")
print(f"实时天气：{weather_info}\n\n")

# 使用函数示例
city = '110101'  # 城市adcode，例如北京市东城区或110101
weather_info = get_amap_weather(city, extensions="all")
print(f"预报天气：{weather_info}\n\n")

实时天气：东城区的实时天气更新于2024-04-05 23:33:31。当前天气晴，温度为14度，西南风，风力≤3级，相对湿度61%。


预报天气：东城区的天气预报，更新时间为2024-04-05 23:33:31。未来几天的天气情况如下：
2024-04-05，星期5，白天晴，气温21度，南风1-3级；夜间晴，气温8度，南风1-3级。
2024-04-06，星期6，白天晴，气温25度，西南风1-3级；夜间晴，气温11度，西南风1-3级。
2024-04-07，星期7，白天晴，气温27度，东北风1-3级；夜间多云，气温12度，东北风1-3级。
2024-04-08，星期1，白天多云，气温23度，南风1-3级；夜间多云，气温12度，南风1-3级。




### 定义一个执行函数execute_function_call

In [4]:
def execute_function_call(message):
    """执行函数调用"""
    
    # 判断功能调用的名称是否为 "get_current_weather"
    if message["function_call"]["name"] == "get_current_weather":
        # 如果是，则获取功能调用的参数
        city_name = json.loads(message["function_call"]["arguments"])["location"]
        # 使用 get_city_adcode 函数获取城市编码
        city_code = get_city_adcode(city_name)
        # 使用 get_amap_weather 函数获取当前天气信息
        weather_info = get_amap_weather(city_code, extensions="base")
        results = weather_info

    # 判断功能调用的名称是否为 "get_n_day_weather_forecast"
    elif message["function_call"]["name"] == "get_n_day_weather_forecast":
        # 如果是，则获取功能调用的参数
        city_name = json.loads(message["function_call"]["arguments"])["location"]
        # 使用 get_city_adcode 函数获取城市编码
        city_code = get_city_adcode(city_name)
        # 使用 get_amap_weather 函数获取未来天气信息
        weather_info = get_amap_weather(city_code, extensions="all")
        results = weather_info

    else:
        # 如果功能调用的名称不是 "get_current_weather" 或 "get_n_day_weather_forecast"，则返回错误信息
        results = f"Error: function {message['function_call']['name']} does not exist"

    # 返回结果
    return results

In [5]:
# 使用了retry库，指定在请求失败时的重试策略。
# 这里设定的是指数等待（wait_random_exponential），时间间隔的最大值为40秒，并且最多重试3次（stop_after_attempt(3)）。
# 定义一个函数chat_completion_request，主要用于发送 聊天补全 请求到OpenAI服务器
@retry(wait=wait_random_exponential(multiplier=1, max=40), stop=stop_after_attempt(3))
def chat_completion_request(messages, functions=None, function_call=None, model=GPT_MODEL):

    # 设定请求的header信息，包括 API_KEY
    headers = {
        "Content-Type": "application/json",
        "Authorization": "Bearer " + os.getenv("OPENAI_API_KEY"),
    }

    # 设定请求的JSON数据，包括GPT 模型名和要进行补全的消息
    json_data = {"model": model, "messages": messages}

    # 如果传入了functions，将其加入到json_data中
    if functions is not None:
        json_data.update({"functions": functions})

    # 如果传入了function_call，将其加入到json_data中
    if function_call is not None:
        json_data.update({"function_call": function_call})

    # 尝试发送POST请求到OpenAI服务器的chat/completions接口
    try:
        response = requests.post(
            "https://api.openai.com/v1/chat/completions",
            headers=headers,
            json=json_data,
        )
        # 返回服务器的响应
        return response

    # 如果发送请求或处理响应时出现异常，打印异常信息并返回
    except Exception as e:
        print("Unable to generate ChatCompletion response")
        print(f"Exception: {e}")
        return e

In [6]:
# 定义一个函数pretty_print_conversation，用于打印消息对话内容
def pretty_print_conversation(messages):

    # 为不同角色设置不同的颜色
    role_to_color = {
        "system": "red",
        "user": "green",
        "assistant": "blue",
        "function": "magenta",
    }

    # 遍历消息列表
    for message in messages:

        # 如果消息的角色是"system"，则用红色打印“content”
        if message["role"] == "system":
            print(colored(f"system: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"user"，则用绿色打印“content”
        elif message["role"] == "user":
            print(colored(f"user: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，并且消息中包含"function_call"，则用蓝色打印"function_call"
        elif message["role"] == "assistant" and message.get("function_call"):
            print(colored(f"assistant[function_call]: {message['function_call']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"assistant"，但是消息中不包含"function_call"，则用蓝色打印“content”
        elif message["role"] == "assistant" and not message.get("function_call"):
            print(colored(f"assistant[content]: {message['content']}\n", role_to_color[message["role"]]))

        # 如果消息的角色是"function"，则用品红色打印“function”
        elif message["role"] == "function":
            print(colored(f"function ({message['name']}): {message['content']}\n", role_to_color[message["role"]]))


In [7]:
# 定义一个名为functions的列表，其中包含两个字典，这两个字典分别定义了两个功能的相关参数

# 第一个字典定义了一个名为"get_current_weather"的功能
functions = [
    {
        "name": "get_current_weather",  # 功能的名称
        "description": "Get the current weather",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
            },
            "required": ["location", "format"],  # 该功能需要的必要参数
        },
    },
    # 第二个字典定义了一个名为"get_n_day_weather_forecast"的功能
    {
        "name": "get_n_day_weather_forecast",  # 功能的名称
        "description": "Get an N-day weather forecast",  # 功能的描述
        "parameters": {  # 定义该功能需要的参数
            "type": "object",
            "properties": {  # 参数的属性
                "location": {  # 地点参数
                    "type": "string",  # 参数类型为字符串
                    "description": "The city and state, e.g. San Francisco, CA",  # 参数的描述
                },
                "format": {  # 温度单位参数
                    "type": "string",  # 参数类型为字符串
                    "enum": ["celsius", "fahrenheit"],  # 参数的取值范围
                    "description": "The temperature unit to use. Infer this from the users location.",  # 参数的描述
                },
                "num_days": {  # 预测天数参数
                    "type": "integer",  # 参数类型为整数
                    "description": "The number of days to forecast",  # 参数的描述
                }
            },
            "required": ["location", "format", "num_days"]  # 该功能需要的必要参数
        },
    },
]

#### 查询北京当天的天气情况

In [11]:
# 定义一个空列表messages，用于存储聊天的内容
messages = []

# 使用append方法向messages列表添加一条系统角色的消息
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."  # 消息的内容
})

# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "I live in Beijing.What's the weather like today"  # 用户询问今天的天气情况
})

# 遍历messages列表并打印每个消息
for message in messages:
    print(f"Role: {message['role']}")
    print(f"Content: {message['content']}")
    print()  # 打印空行以便分隔消息

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 如果助手的消息中有功能调用
if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)


Role: system
Content: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

Role: user
Content: I live in Beijing.What's the weather like today

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: I live in Beijing.What's the weather like today

assistant[function_call]: {'name': 'get_current_weather', 'arguments': '{"location":"Beijing","format":"celsius"}'}

function (get_current_weather): 北京市的实时天气更新于2024-04-05 23:33:30。当前天气晴，温度为14度，西南风，风力≤3级，相对湿度61%。



#### 预报上海3天后的天气情况，高德天气API似乎只支持3天的预报？

In [12]:
# 初始化一个空的messages列表
messages = []

# 向messages列表添加一条系统角色的消息，要求不做关于函数参数值的假设，如果用户的请求模糊，应该寻求澄清
messages.append({
    "role": "system",  # 消息的角色是"system"
    "content": "Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous."
})


# 向messages列表添加一条用户角色的消息
messages.append({
    "role": "user",  # 消息的角色是"user"
    "content": "I live in Shanghai, China.what is the weather going to be like in future？"
})

# 遍历messages列表并打印每个消息
for message in messages:
    print(f"Role: {message['role']}")
    print(f"Content: {message['content']}")
    print()  # 打印空行以便分隔消息

# 使用定义的chat_completion_request函数发起一个请求，传入messages和functions作为参数
chat_response = chat_completion_request(
    messages, functions=functions
)

# 解析返回的JSON数据，获取助手的回复消息
assistant_message = chat_response.json()["choices"][0]["message"]

# 将助手的回复消息添加到messages列表中
messages.append(assistant_message)

# 如果助手的消息中有功能调用
if assistant_message.get("function_call"):
    # 使用 execute_function_call 函数执行功能调用，并获取结果
    results = execute_function_call(assistant_message)
    # 将功能的结果作为一个功能角色的消息添加到消息列表中
    messages.append({"role": "function", "name": assistant_message["function_call"]["name"], "content": results})

# 使用 pretty_print_conversation 函数打印对话
pretty_print_conversation(messages)


Role: system
Content: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

Role: user
Content: I live in Shanghai, China.what is the weather going to be like in future？

system: Don't make assumptions about what values to plug into functions. Ask for clarification if a user request is ambiguous.

user: I live in Shanghai, China.what is the weather going to be like in future？

assistant[function_call]: {'name': 'get_n_day_weather_forecast', 'arguments': '{"location":"Shanghai, China","format":"celsius","num_days":7}'}

function (get_n_day_weather_forecast): 上海市的天气预报，更新时间为2024-04-05 23:31:33。未来几天的天气情况如下：
2024-04-05，星期5，白天晴，气温18度，东北风1-3级；夜间小雨，气温12度，东北风1-3级。
2024-04-06，星期6，白天小雨，气温16度，北风1-3级；夜间晴，气温12度，北风1-3级。
2024-04-07，星期7，白天小雨，气温16度，北风1-3级；夜间小雨，气温13度，北风1-3级。
2024-04-08，星期1，白天小雨，气温18度，北风1-3级；夜间阴，气温12度，北风1-3级。

